# basic idea
- 기존 ML 기반 클러스터링은 잘 작동하지 않았음
- 텍스트가 주제 분류될 때 복잡한 과정을 거치기 때문 ex) KT wiz와 삼성 라이온즈의 맞대결 승자는? &rarr; 스포츠; 하지만 KT와 삼성이라는 단어 때문에 경제로 분류될 가능성 있음.
- 복잡한 로직의 처리를 LLM으로 하면 어떨까? &rarr; LLM 사용 시 출력 컨트롤이 관건
    1. 출력이 영어로 되는 경우 - 프롬프트에서 한국어 명시
    2. 출력이 균일하게 되지 않는 경우 - seed와 temperature 관리
    3. 출력에 노이즈가 생기는 경우 - 노이즈 양에 따라 허용 or 무시

In [1]:
import torch
import re
import pandas as pd
import os

from filter import SpecialCharFilter

from langchain_ollama import ChatOllama
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

In [2]:
df: pd.DataFrame = pd.read_csv('data/train.csv')

special_char_filter = SpecialCharFilter()

noise_df = special_char_filter.filter_noise(df)
clean_df = df[~df.index.isin(noise_df.index)]
print(len(noise_df), len(clean_df))
noise_df.head()

1595 1205


,ID,text,target,special_char_count,special_char_ratio
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,6,0.187500
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,7,0.259259
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,5,0.227273
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,5,0.178571
6,ynat-v1_train_00006,프로야구~롯TKIAs광주 경기 y천취소,1,3,0.142857


In [3]:
prompt = ChatPromptTemplate.from_messages(
    messages=[
        (
            "system",
            """
            You are a helpful assistant that categories news article titles to propre sections. 
            only say in a short korean word.
            """,
        ),
        (
            "human",
            "{input}",
        )
    ]
)

llm = ChatOllama(
    model="gemma2:27b",
    seed=42,
    temperature=0
)

chain = prompt | llm

In [4]:
cluster_path = "data/clustered.csv"

if not os.path.exists(cluster_path):
    for idx, row in clean_df.iterrows():
        input_text = row['text']
        ai_msg = chain.invoke({"input": input_text})
        predicted_label = ai_msg.content.strip()  # 결과 문자열에서 공백 제거
        clean_df.loc[idx, 'predict_label'] = re.sub(r'[^가-힣a-zA-Z\s]', '', predicted_label)
        print(f"Index: {idx}, Input: {input_text}, Predicted Label: {predicted_label}")
else:
    clean_df = pd.read_csv(cluster_path)

위 실행 결과를 보면, 다양한 label로 분류되는 걸 알 수 있다. 따라서 label 텍스트 자체를 임베딩해 클러스터링을 시도한다.

# 실험 1: BERT embedding + K-Means

In [5]:
clean_df.to_csv('data/clustered.csv')

In [6]:
label_counts = clean_df['predict_label'].value_counts()
label_counts.head(10)

predict_label
경제       186
정치       154
스포츠       63
기술        61
사회        59
스포츠       54
국제        45
날씨        35
국제 정치     32
과학        30
Name: count, dtype: int64

klue/bert-base 모델을 활용해 토큰화 및 임베딩 후 k-means 알고리즘으로 클러스터링을 한다.

In [7]:
model_name = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

2024-11-06 07:51:29.338755: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-06 07:51:29.353275: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730847089.378485  494175 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730847089.385646  494175 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 07:51:29.411454: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with t

In [8]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=32, padding="max_length")
    outputs = model(**inputs)
    # [CLS] 토큰의 출력만 사용하여 임베딩 생성
    embedding = outputs.last_hidden_state[:, 0, :].squeeze().detach().numpy()
    return embedding

In [9]:
# embeddings = clean_df['predict_label'].astype(str).apply(get_embedding).tolist()
# embeddings = torch.tensor(embeddings)  # 리스트를 텐서로 변환

# # KMeans 클러스터링 (7개의 클러스터로 설정)
# kmeans = KMeans(n_clusters=7, random_state=42)
# clean_df['label_cluster'] = kmeans.fit_predict(embeddings)

# # 클러스터링 결과 확인
# print(clean_df[['predict_label', 'label_cluster']].head())

In [10]:
# cluster_counts = clean_df['label_cluster'].value_counts()
# print("\nCluster distribution:")
# print(cluster_counts)

In [11]:
# cross_tab = pd.crosstab(clean_df['label_cluster'], clean_df['target'])
# cross_tab

클러스터링 실험 결과 사전에 분석한 라벨 분포와 다르다. 사전 분석 시 라벨이 골고루 분포되어 있었다.

# 실험 2: LLM 라벨 예측 결과 분류 - 대화 정보 기억

LLM의 출력 결과를 7가지로 추리기 위해서 어떤 기능을 사용해야 할까?
LLM agent에서 버퍼 메모리 기능 사용해보기

In [12]:
store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain = RunnableWithMessageHistory(llm, get_session_history)

In [13]:
def extract_categories(titles, session_id):
    prompt = (
        "다음은 뉴스 기사 제목들의 목록입니다:\n\n"
        + "\n".join(titles)
        + "\n\n이 제목들을 7개의 대표적인 카테고리로 분류하여, 각 카테고리의 이름만 한 단어로 출력하세요."
    )
    response = chain.invoke(
        prompt,
        config={"configurable": {"session_id": session_id}},
    )
    return response.content.strip()

def classify_title(title, categories, session_id):
    prompt = (
        f"다음은 뉴스 기사 제목입니다:\n\n"
        f"제목: {title}\n\n"
        f"반드시 아래의 카테고리 중 하나로 분류하고, 해당 카테고리의 이름만 한 단어로 출력하세요:\n{', '.join(categories)}"
    )
    response = chain.invoke(
        prompt,
        config={"configurable": {"session_id": session_id}},
    )
    return response.content.strip()

In [14]:
# 예시 기사 제목 리스트
titles = clean_df['text'].to_list()

# 세션 ID 설정
session_id = "1"

if not os.path.exists('data/clustered_7_labels.csv'):
    # 1. 카테고리 추출
    categories = extract_categories(titles, session_id)
    print("추출된 카테고리:\n", categories)

    # 2. 각 제목에 대한 분류
    classified_categories = []
    for title in tqdm(titles):
        category = classify_title(title, categories, session_id)
        classified_categories.append(category)
        print(f"제목: {title}\n분류된 카테고리: {category}\n")
        
    clean_df['label_cluster'] = classified_categories
    clean_df.to_csv('data/clustered_7_labels.csv')
else:
    clean_df = pd.read_csv('data/clustered_7_labels.csv')      

백그라운드 실행 명령어
```bash
nohup ollama serve &
nohup jupyter nbconvert --to notebook --execute LLM_based_clustering.ipynb --output LLM_based_clustering_output.ipynb &
```

In [15]:
label_count = clean_df['label_cluster'].value_counts()
label_count

label_cluster
경제                                          272
스포츠                                         179
정치                                          177
국제                                          160
문화                                          159
사회                                          140
과학                                          102
날씨                                            5
교육                                            4
역사                                            3
학문                                            1
환경                                            1
게임 \n\n(배틀그라운드는 게임입니다.)                       1
정치 \n\n(제목이 정치적인 내용을 담고 있음을 추측할 수 있습니다.)      1
Name: count, dtype: int64

In [16]:
most_labels = label_count.head(7).index.to_list()
extra = clean_df[~clean_df['label_cluster'].isin(most_labels)]
extra

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ID,text,target,special_char_count,special_char_ratio,predict_label,label_cluster
62,62,62,62,62,161,ynat-v1_train_00161,S9XOC카cO트6온라인 F개강좌 공동 개설,3,1,0.041667,교육,교육
76,76,76,76,76,189,ynat-v1_train_00189,400년 전 일본 건너간 조선 첫 여성도공 백파선,1,0,0.000000,역사,역사
125,125,125,125,125,288,ynat-v1_train_00288,게시판 제38회 한국출판학회상에 천재교육 등 선정,4,0,0.000000,교육,학문
150,150,150,150,150,331,ynat-v1_train_00331,섬들의 근대화는 달랐다 일제가 쓰고 버린 태평양 섬 이야기,6,0,0.000000,역사,역사
203,203,203,203,203,426,ynat-v1_train_00426,전북 주요 대학들 등록금 동결 학부모 부담 덜겠다,1,0,0.000000,교육,교육
239,239,239,239,239,536,ynat-v1_train_00536,제1차 세계대전 종전은 새로운 폭력의 시작이었다,2,0,0.000000,역사,역사
243,243,243,243,243,545,ynat-v1_train_00545,대관령 영하 18.3도 나흘째 맹추위 동해안 건조특보,1,0,0.000000,날씨,날씨
342,342,342,342,342,795,ynat-v1_train_00795,내일날씨 제주도 영동 아침까지 비 동해 남해 풍랑 주의,3,0,0.000000,날씨,날씨
358,358,358,358,358,857,ynat-v1_train_00857,교육의 틀 바꾼다 충북교육청 학사운영 등 대대적 재구조화,3,0,0.000000,교육,교육
505,505,505,505,505,1241,ynat-v1_train_01241,제주 전 학교 등교 중지 내년 1월 3일까지 연장 원격수업 시행,3,0,0.000000,교육,교육


In [17]:
extra['label_cluster'] = extra['label_cluster'].apply(lambda x: classify_title(x, most_labels, session_id))
extra

/tmp/ipykernel_494175/2926970596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extra['label_cluster'] = extra['label_cluster'].apply(lambda x: classify_title(x, most_labels, session_id))


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ID,text,target,special_char_count,special_char_ratio,predict_label,label_cluster
62,62,62,62,62,161,ynat-v1_train_00161,S9XOC카cO트6온라인 F개강좌 공동 개설,3,1,0.041667,교육,사회
76,76,76,76,76,189,ynat-v1_train_00189,400년 전 일본 건너간 조선 첫 여성도공 백파선,1,0,0.000000,역사,문화
125,125,125,125,125,288,ynat-v1_train_00288,게시판 제38회 한국출판학회상에 천재교육 등 선정,4,0,0.000000,교육,과학
150,150,150,150,150,331,ynat-v1_train_00331,섬들의 근대화는 달랐다 일제가 쓰고 버린 태평양 섬 이야기,6,0,0.000000,역사,문화
203,203,203,203,203,426,ynat-v1_train_00426,전북 주요 대학들 등록금 동결 학부모 부담 덜겠다,1,0,0.000000,교육,사회
239,239,239,239,239,536,ynat-v1_train_00536,제1차 세계대전 종전은 새로운 폭력의 시작이었다,2,0,0.000000,역사,문화
243,243,243,243,243,545,ynat-v1_train_00545,대관령 영하 18.3도 나흘째 맹추위 동해안 건조특보,1,0,0.000000,날씨,사회
342,342,342,342,342,795,ynat-v1_train_00795,내일날씨 제주도 영동 아침까지 비 동해 남해 풍랑 주의,3,0,0.000000,날씨,사회
358,358,358,358,358,857,ynat-v1_train_00857,교육의 틀 바꾼다 충북교육청 학사운영 등 대대적 재구조화,3,0,0.000000,교육,사회
505,505,505,505,505,1241,ynat-v1_train_01241,제주 전 학교 등교 중지 내년 1월 3일까지 연장 원격수업 시행,3,0,0.000000,교육,사회


In [18]:
label_count = extra['label_cluster'].value_counts()
label_count

label_cluster
사회    9
문화    4
과학    2
정치    1
Name: count, dtype: int64

In [20]:
clean_df.update(extra[['label_cluster']])

columns_to_drop = ['Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0', 'Unnamed: 0.1' ,
                   'special_char_count', 'special_char_ratio', 'predict_label']
clean_df.drop(columns=columns_to_drop, inplace=True)

clean_df

,ID,text,target,label_cluster
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,5,경제
1,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자 연인 빚 떠안은 적 있다,0,사회
2,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파 3위 굳,4,스포츠
3,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요 경주 지진현장 방문종합,6,정치
4,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔 알뜰폰 기대감,4,경제
...,...,...,...,...
1200,ynat-v1_train_02794,문 대통령 김기식 금감원장 사표 수리키로종합,2,정치
1201,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨 10점만점에 12점도,6,국제
1202,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2,경제
1203,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해 승부는 냉정,1,스포츠


In [21]:
label_count = clean_df['label_cluster'].value_counts()
label_count

label_cluster
경제     272
스포츠    179
정치     178
문화     163
국제     160
사회     149
과학     104
Name: count, dtype: int64